# Extending PCA to higher dimensions

In [6]:
!pip install -q otter-grader

import otter
grader = otter.Notebook("hw7.ipynb")

import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'
from sklearn.decomposition import PCA
# f

In this assignment, we will extend PCA to cases of more than just two input dimensions.

**You will need to refer to the class lab notebook on PCA to complete this assignment.**

The following cell loads the same face rating data that we worked with last time:

In [7]:
ratings = pd.read_csv('face_ratings.csv')
stim = ratings['stimulus'].copy()
ratings = ratings.drop(columns=['stimulus']).apply(zscore)
ratings['stimulus'] = stim
ratings.head()

,aggressive,attractive,caring,confident,dominant,emot_stable,intelligent,mean,responsible,sociable,trustworthy,unhappy,weird,stimulus
0,-0.838576,2.150772,1.299035,1.241495,-0.240222,1.313222,0.881080,-0.485750,1.863745,1.670497,2.050984,-0.796445,-1.174697,AF01NES
1,-0.416323,0.112142,0.448366,-1.203330,-0.153719,-0.532231,-0.163642,-0.066824,-0.398276,-0.374984,-0.197564,0.961431,0.271219,AF02NES
2,-0.859174,-0.225465,1.006182,0.337651,-1.241176,-0.035868,-1.075934,-0.532297,0.056728,0.288781,0.278320,-0.369184,0.656796,AF03NES
3,-0.395726,-0.653967,0.016058,-0.581011,-0.437941,-0.264958,-0.987648,0.631387,-0.541278,-0.645909,-0.042902,0.692867,0.830306,AF04NES
4,-1.528599,0.501689,1.173527,0.219114,-0.833380,0.740495,0.733936,-0.776671,0.953736,0.830630,1.146806,-1.309159,-0.239671,AF05NES


Recall that so far we've been working with just two the 13 dimensions:

In [8]:
x = ratings['responsible'].values
y = ratings['trustworthy'].values

We previously defined the first principal component as the line that maximizes variance explained by the points projected onto it.

Another way to define the first principal component is as the line that minimizes the sum of squared distances between the original points and their projections.

Note that the slope we found before was approximately 1. We'd expect to find the same slope when minimizes the sum of squared distances.

**Question 1:** Find the slope (in `slopes_to_try`) that minimizes the sum of squared distances between the original points and their projections. Store the sum for each candidate slope in a numpy array called `sum_per_slope`. Store the best slope in `best_slope_min_distance`.

In [9]:
# do not change
slopes_to_try = np.linspace(0, 2, 200)

# do not change
def project_point_onto_line(x, y, slope):
  x_p = (x + slope*y) / (slope**2 + 1)
  y_p = x_p * slope
  return x_p, y_p

# Your code here

sum_per_slope = np.array([
    np.sum((x - project_point_onto_line(x, y, slope)[0])**2 +
           (y - project_point_onto_line(x, y, slope)[1])**2)
    for slope in slopes_to_try
])

best_slope_min_distance = slopes_to_try[np.argmin(sum_per_slope)]

# do not change
best_slope_min_distance

np.float64(1.0050251256281406)

In [10]:
grader.check("q1")

q1 results: All test cases passed!

Let's extend the process above to three dimensions ($x$, $y$, $z$) to understand how responsible, trustworthy, and now **caring** ratings relate to one another.

First we can load our third dimension:

In [11]:
z = ratings['caring'].values

We can see these points in 3D space using the plot below. **Click and drag** your mouse to **rotate** the plot below and see how the points are related to each other in the space.

In [15]:
fig = go.Figure()

# x, y, z data points in 3D
fig.add_trace(go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=4,opacity=0.7, color='blue'),
    name='Original Data'
))

# Formatting
fig.update_layout(
    scene=dict(
        xaxis_title='Responsible',
        yaxis_title='Trustworthy',
        zaxis_title='Caring',
        aspectmode='cube',
    ),
    width=800, height=800,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)
fig.show()

All three sets of ratings are clearly correlated. We can further quantify what we see by computing correlation coefficients:

In [13]:
print('Correlation between responsible and trustworthy dimensions:', np.corrcoef(x, y)[0, 1])
print('Correlation between responsible and caring dimensions:', np.corrcoef(x, z)[0, 1])
print('Correlation between caring and trustworthy dimensions:', np.corrcoef(z, y)[0, 1])

Correlation between responsible and trustworthy dimensions: 0.8833615858402782
Correlation between responsible and caring dimensions: 0.766116004115042
Correlation between caring and trustworthy dimensions: 0.8311240682181321


Importantly, the correlations are not perfect, and the points are distributed across all three dimensions of the space to some extent.

Nonetheless, as in the original 2D case, it's reasonable to suspect that a single dimension may capture all three existing dimensions to a large extent (even if not fully).

So far, we've explored the application of PCA to two dimensions where we searched for a single slope $\text{m}$ to define a line that maximizes the variance of projected points. That is, our task has been to find a single dimension that explains as much of the variation in ratings as possible. 

Let's extend this idea to three dimensions ($x$, $y$, $z$).

First, we need to recompute total variance with variation in "caring" ratings included.

**Question 2:** Compute `total_variance` for all three rating dimensions.

In [16]:
# Your code here
total_variance = np.var(x) + np.var(y) + np.var(z)

# do not change
total_variance

np.float64(3.0)

In [17]:
grader.check("q2")

q2 results: All test cases passed!



Our goal is still to find a single dimension (a line; PC1) that captures as much of this total variance as possible. In 3D space, lines are defined by **two** slopes (`n_dimensions - 1`) rather than one:
- $\text{m}_{\text{xy}}$ (relating trustworthy to responsible)
- $\text{m}_{\text{xz}}$ (relating caring to responsible)

Our line equation then becomes:
- $y = \text{m}_{\text{xy}} x$
- $z = \text{m}_{\text{xz}} x.$

In two dimensions, we varied the slope parameter $\text{m}$ and projected points onto the line defined by that slope. Similarly, in three dimensions, we want to vary $\text{m}_{\text{xy}}$ and $\text{m}_{\text{xz}}$, and project points onto the line defined by those slopes.

To project points onto a line in 3D, we can extend our original projection formula from:
$$x_p = (x + \text{m}y) / (\text{m}^2 + 1)$$
to:
$$x_p = (x + \text{m}_{\text{xy}}y + \text{m}_{\text{xz}}z) / (\text{m}_{\text{xy}}^2 + \text{m}_{\text{xz}}^2 + 1).$$

Hopefully you can see from the pattern that the projection formula can be generalized to any number of dimensions.

Just as before, once we obtain $x_p$, we can easily compute $y_p = \text{m}_{\text{xy}} x_p$ and $z_p = \text{m}_{\text{xz}} x_p$.

**Question 3:** Create a function called `project_point_onto_line_3d` that takes `x`, `y`, `z`, `slope_xy`, and `slope_xz` and returns `x_p`, `y_p`, and `z_p`.

In [18]:
# Your code here
def project_point_onto_line_3d(x, y, z, slope_xy, slope_xz):
    x_p = (x + slope_xy * y + slope_xz * z) / (1 + slope_xy**2 + slope_xz**2)
    y_p = slope_xy * x_p
    z_p = slope_xz * x_p
    return x_p, y_p, z_p


In [19]:
grader.check("q3")

q3 results: All test cases passed!

Now, we can find PC1 by finding the best **pair** of slopes that minimizes the sum of squared distances between the original points and their projections.

**Question 4:** Create a function called `find_best_slopes` that takes as input `x`, `y`, and `z` and returns as output `best_slope_xy`, `best_slope_xz`, and `min_distance_sum`. Unlike the single-slope case, you will need to make use of a nested loop to find the best slope values. For both, search only the slopes in `np.linspace(-1.4, 1.4, 500)`.

In [20]:
# Your code here
def find_best_slopes(x, y, z):
    min_distance_sum = float('inf')
    best_slope_xy = None
    best_slope_xz = None
    
    for slope_xy in np.linspace(-1.4, 1.4, 500):
        for slope_xz in np.linspace(-1.4, 1.4, 500):
            x_p = (x + slope_xy * y + slope_xz * z) / (1 + slope_xy**2 + slope_xz**2)
            y_p = slope_xy * x_p
            z_p = slope_xz * x_p
            distance_sum = np.sum((x - x_p)**2 + (y - y_p)**2 + (z - z_p)**2)
            if distance_sum < min_distance_sum:
                min_distance_sum = distance_sum
                best_slope_xy = slope_xy
                best_slope_xz = slope_xz
                
    return best_slope_xy, best_slope_xz, min_distance_sum


# do not change
best_slope_xy, best_slope_xz, min_distance_sum = find_best_slopes(x, y, z)

In [21]:
grader.check("q4")

q4 results: All test cases passed!

We can see the best pair of slopes and correponding minimum sum:

In [22]:
print(f"Best slopes that minimize the sum of squared distances:")
print(f"slope_xy = {best_slope_xy}, slope_xz = {best_slope_xz}")
print(f"Sum of squared distances: {min_distance_sum}")

Best slopes that minimize the sum of squared distances:
slope_xy = 1.0240480961923848, slope_xz = 0.9791583166332667
Sum of squared distances: 22.792760784022335


As in the 2D case, we can now project our points onto PC1:

In [23]:
x_p, y_p, z_p = project_point_onto_line_3d(x, y, z, best_slope_xy, best_slope_xz)

We can then visualize the projection of these points onto PC1 below. **Click and drag** your mouse to **rotate** the plot below and visualize the projection.

Note that the blue dots are the original points, the red line is PC1, and the blue lines are the shortest paths between the original points and their projections onto PC1.

In [24]:
fig = go.Figure()
# Add original data points
fig.add_trace(go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=4,opacity=0.7, color='blue'),
    name='Original Data'
))
# Range for pc1 line
data_range = max(np.max(x) - np.min(x), np.max(y) - np.min(y), np.max(z) - np.min(z))
# PC1 endpoints
line_dir = np.array([1.0, best_slope_xy, best_slope_xz])
line_dir = line_dir / np.sqrt(np.sum(line_dir**2))  # normalize
# Scale to cover the data range
t_values = np.linspace(-data_range, data_range, 100)
line_points = np.outer(t_values, line_dir)
# PC1 line
fig.add_trace(go.Scatter3d(
    x=line_points[:, 0], 
    y=line_points[:, 1], 
    z=line_points[:, 2],
    mode='lines',
    line=dict(color='red', width=6),
    name='PC1'
))
# Projection lines
for i in range(0, len(x)):
    fig.add_trace(go.Scatter3d(
        x=[x[i], x_p[i]],
        y=[y[i], y_p[i]],
        z=[z[i], z_p[i]],
        mode='lines',
        line=dict(color='blue', width=1),
        showlegend=False
    ))
# Formatting
fig.update_layout(
    scene=dict(
        xaxis_title='Responsible',
        yaxis_title='Trustworthy',
        zaxis_title='Caring',
        aspectmode='cube',
    ),
    width=800, height=800,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)
fig.show()

As before in the 2D case, we will need to compute variance explained by PC1. Recall that to do that, we need to compute variance along PC1 and then divide by the total variance. Our formula for finding the positions of points along our new dimension now changes from the 2D case: $\text{sign}{(x_p)} \times \sqrt{x_p^2 + y_p^2}$, to the 3D case: $\text{sign}{(x_p)} \times \sqrt{x_p^2 + y_p^2 + z_p^2}$.

**Question 5:** Create a function called `compute_variance_along_line_in_3d` that takes as input `x`, `y`, and `z`, and outputs `variance_along_line`.

In [26]:
# Your code here
def compute_variance_along_line_in_3d(x, y, z):
    slope_xy, slope_xz, _ = find_best_slopes(x, y, z)
    x_p, y_p, z_p = project_point_onto_line_3d(x, y, z, slope_xy, slope_xz)
    projections_1d = np.sign(x_p) * np.sqrt(x_p**2 + y_p**2 + z_p**2)
    variance_along_line = np.var(projections_1d)
    return variance_along_line


# do not change
variance_along_PC1 = compute_variance_along_line_in_3d(x_p, y_p, z_p)
variance_explained_by_PC1 = variance_along_PC1 / total_variance
print(f"Variance explained by PC1: {variance_explained_by_PC1:.4f}")

Variance explained by PC1: 0.8849


In [27]:
grader.check("q5")

q5 results: All test cases passed!

Note that ~88% of the total variance is explained by a single dimension (PC1).

We can now move on to finding PC2. Recall that PC2 is the line that maximizes variance (and now equivalently, minimizes squared distance) for the residual variation after removing PC1.

**Question 6:** Create a function called `compute_residuals_3d` that takes as input `x`, `y`, `z`, `x_p`, `y_p`, `z_p` and returns as output `x_res`, `y_res`, `z_res`.

In [28]:
# Your code here
def compute_residuals_3d(x, y, z, x_p, y_p, z_p):
    x_res = x - x_p
    y_res = y - y_p
    z_res = z - z_p
    return x_res, y_res, z_res


In [29]:
grader.check("q6")

q6 results: All test cases passed!

We can now apply our function to compute residuals:

In [30]:
# what's left after removing PC1 values
x_res, y_res, z_res = compute_residuals_3d(x, y, z, x_p, y_p, z_p)

In the below plot, the residuals are plotted as green points.

Move the plot around and notice that the green points fall on a 2D plane. Why? Recall that by computing residuals, we are removing variation along PC1. Note that unlike the blue points, the green points do not vary along PC1. Instead, their value on PC1 is held constant at 0. Removing variability along one dimension means these points now fall along a plane.

In [31]:
fig = go.Figure()

# Add original data points (blue)
fig.add_trace(go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=4, opacity=0.7, color='blue'),
    name='Original data'
))

# Add PC1 line (red)
line_dir = np.array([1.0, best_slope_xy, best_slope_xz])
line_dir = line_dir / np.sqrt(np.sum(line_dir**2))  # normalize
t_values = np.linspace(-data_range*1.2, data_range*1.2, 100)
line_points = np.outer(t_values, line_dir)

fig.add_trace(go.Scatter3d(
    x=line_points[:, 0], 
    y=line_points[:, 1], 
    z=line_points[:, 2],
    mode='lines',
    line=dict(color='red', width=6),
    name=f'PC1'
))

# Add residual points (green)
fig.add_trace(go.Scatter3d(
    x=x_res, y=y_res, z=z_res,
    mode='markers',
    marker=dict(size=4, opacity=0.7, color='green'),
    name='Residuals'
))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='Responsible',
        yaxis_title='Trustworthy',
        zaxis_title='Caring',
        aspectmode='cube'
    ),
    width=1000, height=1000,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

After removing the variability we've already explained using PC1 (which was most of the total variance in the data), there's still some visible remaining variability across the 2D plane of green points.

To complete the PCA analysis, we'd find a second dimension (PC2) to explain most of the remaining variance (i.e., in the green points), giving us two new slopes.

If we were to then compute new residuals after removing variation along PC2, our points would then lie along a line rather than a plane. This final line would be a **third** principal component.

In general, given M input dimensions, there will be M principal components. Whereas each of the original dimensions explains the same amount of variance, the first principal component explains the most variance, the second explains the second most, and so on.

We can perform PCA for the entire 13-dimensional set of ratings using `scikit-learn` and print out the varianced explained values for all 13 principal components:

In [32]:
pca = PCA()
ratings_matrix = ratings.drop(columns=['stimulus']).values
pca.fit(ratings_matrix)

explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.62551994, 0.18460995, 0.06174318, 0.05031116, 0.02113982,
       0.01500076, 0.01225332, 0.00795414, 0.00667045, 0.00453747,
       0.00393348, 0.00326147, 0.00306487])

Notice that a single dimension (PC1) explains ~63% of the total variance. Previous work has shown that this dimension tends to capture a general shared dimension of "social dependability" (trustworthy, repsonsible, caring, etc) much like what we found in our 2D analysis. PC2 explains ~18%. Previous work has shown that this dimension tends to capture a general shared dimension of "dominance" (competence, aggressiveness, etc).